In [1]:
import pandas as pd
import numpy

In [2]:
performances = pd.read_csv("performances.csv")
aspects = pd.read_csv("judged-aspects.csv")
scores = pd.read_csv("judge-scores.csv")
judges = pd.read_csv("judges.csv")
country_judge = pd.read_csv("judge-country.csv")

In [3]:
performances.head(1)

,performance_id,competition,program,name,nation,rank,starting_number,total_segment_score,total_element_score,total_component_score,total_deductions
0,6e4bd41ac5,Grand Prix Final 2017 Senior and Junior,ICE DANCE FREE DANCE,Anna CAPPELLINI / Luca LANOTTE,ITA,5,1,110.99,56.01,54.98,0.0


In [4]:
# rinomino le colonne per evitare incomprensioni dopo le join
performances.rename(columns={'name': 'athlete_name',
                             'nation': 'athlete_nation'}, inplace=True)

In [5]:
# verifico la presenza di NA
performances.isnull().sum(axis = 0)

performance_id           0
competition              0
program                  0
athlete_name             0
athlete_nation           0
rank                     0
starting_number          0
total_segment_score      0
total_element_score      0
total_component_score    0
total_deductions         0
dtype: int64

In [6]:
# se le seguenti colonne dovessero servire per rispondere a delle domande, non runnare questa cella
# - starting_number

performances.drop('starting_number', axis=1, inplace=True)
#performances.drop('total_segment_score', axis=1, inplace=True)
#performances.drop('total_element_score', axis=1, inplace=True)
#performances.drop('total_component_score', axis=1, inplace=True)

In [7]:
aspects.head(6)

,aspect_id,performance_id,section,aspect_num,aspect_desc,info_flag,credit_flag,base_value,factor,goe,ref,scores_of_panel
0,00034b9414,b639d77459,components,NaN,Transitions,NaN,NaN,NaN,0.8,NaN,NaN,9.07
1,0004dd08d6,9589bb2ebf,components,NaN,Transitions,NaN,NaN,NaN,0.8,NaN,NaN,6.64
2,000b8b5070,71d2b1b597,components,NaN,Performance,NaN,NaN,NaN,1.0,NaN,NaN,7.68
3,000cec97c9,41a2345881,components,NaN,Transitions,NaN,NaN,NaN,0.8,NaN,NaN,4.71
4,000df5399a,b017147b2f,elements,1.0,3Tw2,NaN,NaN,5.8,NaN,-0.20,NaN,5.60
5,000f259b7c,a4a69ced64,elements,2.0,CoSp4,NaN,NaN,5.6,NaN,0.77,NaN,6.37


In [8]:
# rinomino le colonne per evitare incomprensioni dopo le join
aspects.rename(columns={'scores_of_panel': 'aspect_final_score'}, inplace=True)

In [9]:
# verifico la presenza di NA
print(aspects.isnull().sum(axis = 0))

# elimino le colonne con NA che non mi servono
# - aspect_num      8630 NA (the positional order of the aspect within the performance and section)
# - info_flag
# - credit_flag
# - base_value      8630 NA
# - goe             8630 NA
# - ref
# - factor

# Sospetto che le 8630 NA ci siano per i components.
# Controllo:
print(aspects['section'].value_counts()['components']) #8630 infatti!
print(aspects[aspects["section"] == 'components'].head())

#aspects.drop('aspect_num', axis=1, inplace=True)
aspects.drop('info_flag', axis=1, inplace=True)
aspects.drop('credit_flag', axis=1, inplace=True)
#aspects.drop('base_value', axis=1, inplace=True)
aspects.drop('goe', axis=1, inplace=True)
aspects.drop('ref', axis=1, inplace=True)
aspects.drop('factor', axis=1, inplace=True)



aspect_id                 0
performance_id            0
section                   0
aspect_num             8630
aspect_desc               0
info_flag             22937
credit_flag           21204
base_value             8630
factor                15302
goe                    8630
ref                   23932
aspect_final_score        0
dtype: int64
8630
    aspect_id performance_id     section  aspect_num  aspect_desc info_flag  \
0  00034b9414     b639d77459  components         NaN  Transitions       NaN   
1  0004dd08d6     9589bb2ebf  components         NaN  Transitions       NaN   
2  000b8b5070     71d2b1b597  components         NaN  Performance       NaN   
3  000cec97c9     41a2345881  components         NaN  Transitions       NaN   
6  001282e1b1     012b24851b  components         NaN  Performance       NaN   

  credit_flag  base_value  factor  goe  ref  aspect_final_score  
0         NaN         NaN     0.8  NaN  NaN                9.07  
1         NaN         NaN     0.8  NaN 

In [10]:
# rinomino le colonne per evitare incomprensioni dopo le join

# ATT: goe (-3,+3) is translated by base_value to the actual number of points awarded for that element
aspects.rename(columns={'base_value': 'element_base_value',
                        'aspect_num': 'element_order'}, inplace=True)

In [11]:
scores.head(1)

,aspect_id,judge,score
0,00034b9414,J1,9.0


In [12]:
# rinomino le colonne per evitare incomprensioni dopo le join
scores.rename(columns={'score': 'judge_score'}, inplace=True) #GOE (for elements) and score (for components)!!!!

In [13]:
# verifico la presenza di NA
scores.isnull().sum(axis = 0)

aspect_id      0
judge          0
judge_score    0
dtype: int64

In [14]:
judges.head(1)

,judge_name,assigned_country,role,segment_category,pdf_name,program,competition,officials_table_link,results_pdf_link,clean_judge_name
0,Ms. Chihee RHEE,KOR,Judge No.1,Free Dance|Ice Dance,4f3031488c_data0405.pdf,ICE DANCE FREE DANCE,Grand Prix Final 2017 Senior and Junior,http://www.isuresults.com/results/season1718/g...,http://www.isuresults.com/results/season1718/g...,Chihee RHEE


In [15]:
country_judge.head(1)

,clean_judge_name,judge_country
0,Adriana DOMANSKA,SVK


In [16]:
# faccio la join tra le tabelle judges e country_judge per ottenere la nazionalità dei giudici
print("judges.shape:",judges.shape)
print("country_judge.shape:",country_judge.shape)
judges = judges.merge(country_judge,
                      how='inner',
                      on='clean_judge_name'
                      )
print("judges.shape after join",judges.shape)

judges.shape: (1368, 10)
country_judge.shape: (241, 2)
judges.shape after join (1367, 11)


In [17]:
judges.head(1)

,judge_name,assigned_country,role,segment_category,pdf_name,program,competition,officials_table_link,results_pdf_link,clean_judge_name,judge_country
0,Ms. Chihee RHEE,KOR,Judge No.1,Free Dance|Ice Dance,4f3031488c_data0405.pdf,ICE DANCE FREE DANCE,Grand Prix Final 2017 Senior and Junior,http://www.isuresults.com/results/season1718/g...,http://www.isuresults.com/results/season1718/g...,Chihee RHEE,KOR


In [18]:
# rinomino le colonne per evitare incomprensioni dopo le join
judges.rename(columns={'judge_name': 'del_judge_name',
                       'clean_judge_name': 'judge_name',
                       'judge_country': 'judge_nation'}, inplace=True)

In [19]:
# elimino le colonne inutili
judges.drop('segment_category', axis=1, inplace=True)
judges.drop('pdf_name', axis=1, inplace=True)
judges.drop('officials_table_link', axis=1, inplace=True)
judges.drop('results_pdf_link', axis=1, inplace=True)
judges.drop('del_judge_name', axis=1, inplace=True)
judges.drop('assigned_country', axis=1, inplace=True)

In [20]:
scores['judge'].value_counts()

J1    23843
J2    23843
J3    23843
J4    23843
J5    23843
J7    23843
J8    23843
J9    23843
J6    23787
Name: judge, dtype: int64

In [21]:
judges['role'].value_counts()

Judge No.1    152
Judge No.2    152
Judge No.9    152
Judge No.7    152
Judge No.3    152
Judge No.8    152
Judge No.4    152
Judge No.5    152
Judge No.6    151
Name: role, dtype: int64

In [22]:
# uniformo i nomi dei giudici
judges.loc[judges["role"] == "Judge No.1", "role"] = "J1"
judges.loc[judges["role"] == "Judge No.2", "role"] = "J2"
judges.loc[judges["role"] == "Judge No.3", "role"] = "J3"
judges.loc[judges["role"] == "Judge No.4", "role"] = "J4"
judges.loc[judges["role"] == "Judge No.5", "role"] = "J5"
judges.loc[judges["role"] == "Judge No.6", "role"] = "J6"
judges.loc[judges["role"] == "Judge No.7", "role"] = "J7"
judges.loc[judges["role"] == "Judge No.8", "role"] = "J8"
judges.loc[judges["role"] == "Judge No.9", "role"] = "J9"

In [23]:
# creo il df facendo una inner join tra aspects e performances:
print("performances.shape:",performances.shape)
print("aspects.shape:",aspects.shape)
df = aspects.merge(performances,
                   how='inner',
                   on='performance_id')

performances.shape: (1726, 10)
aspects.shape: (23932, 7)


In [24]:
# aggiungo tutta la tabella scores
print("df.shape:",df.shape)
print("scores.shape:",scores.shape)
df = df.merge(scores,
              how='inner',
              on='aspect_id')

df.shape: (23932, 16)
scores.shape: (214531, 3)


In [25]:
print("df.shape:",df.shape)

df.shape: (214531, 18)


In [26]:
# aggiungo nomi e nazionalità dei giudici
print("df.shape:",df.shape)
print("judges.shape:",judges.shape)
df = df.merge(judges,
              how='inner',
              left_on=['competition','program','judge'],
              right_on=['competition','program','role'])

df.shape: (214531, 18)
judges.shape: (1367, 5)


In [27]:
print("df.shape:",df.shape)

df.shape: (214491, 21)


In [28]:
df.head(1)

,aspect_id,performance_id,section,element_order,aspect_desc,element_base_value,aspect_final_score,competition,program,athlete_name,...,rank,total_segment_score,total_element_score,total_component_score,total_deductions,judge,judge_score,role,judge_name,judge_nation
0,00034b9414,b639d77459,components,NaN,Transitions,NaN,9.07,ISU European Figure Skating Championships 2017,LADIES SHORT PROGRAM,Evgenia MEDVEDEVA,...,1,78.92,42.0,36.92,0.0,J1,9.0,J1,Agita ABELE,LAT


In [29]:
# elimino le colonne non più necessarie
df.drop('judge', axis=1, inplace=True)
df.drop('role', axis=1, inplace=True)

In [30]:
df.head(1)

,aspect_id,performance_id,section,element_order,aspect_desc,element_base_value,aspect_final_score,competition,program,athlete_name,athlete_nation,rank,total_segment_score,total_element_score,total_component_score,total_deductions,judge_score,judge_name,judge_nation
0,00034b9414,b639d77459,components,NaN,Transitions,NaN,9.07,ISU European Figure Skating Championships 2017,LADIES SHORT PROGRAM,Evgenia MEDVEDEVA,RUS,1,78.92,42.0,36.92,0.0,9.0,Agita ABELE,LAT


In [31]:
# riordino le colonne in modo sensato
df = df[['competition',
         'program',
         'performance_id',
         'athlete_name',
         'athlete_nation',
         'rank',
         'total_segment_score',
         'total_element_score',
         'total_component_score',
         'total_deductions',
         'aspect_id',
         'aspect_desc',
         'element_order',
         'element_base_value',
         'section',
         'aspect_final_score',
         'judge_name',
         'judge_nation',
         'judge_score'
        ]]

In [32]:
df.head(20)

,competition,program,performance_id,athlete_name,athlete_nation,rank,total_segment_score,total_element_score,total_component_score,total_deductions,aspect_id,aspect_desc,element_order,element_base_value,section,aspect_final_score,judge_name,judge_nation,judge_score
0,ISU European Figure Skating Championships 2017,LADIES SHORT PROGRAM,b639d77459,Evgenia MEDVEDEVA,RUS,1,78.92,42.00,36.92,0.0,00034b9414,Transitions,NaN,NaN,components,9.07,Agita ABELE,LAT,9.00
1,ISU European Figure Skating Championships 2017,LADIES SHORT PROGRAM,b639d77459,Evgenia MEDVEDEVA,RUS,1,78.92,42.00,36.92,0.0,0127af74a7,Skating Skills,NaN,NaN,components,9.14,Agita ABELE,LAT,9.25
2,ISU European Figure Skating Championships 2017,LADIES SHORT PROGRAM,b639d77459,Evgenia MEDVEDEVA,RUS,1,78.92,42.00,36.92,0.0,38bda018f5,3Lo,4.0,5.61,elements,7.21,Agita ABELE,LAT,3.00
3,ISU European Figure Skating Championships 2017,LADIES SHORT PROGRAM,b639d77459,Evgenia MEDVEDEVA,RUS,1,78.92,42.00,36.92,0.0,50366b5d60,3F+3T,3.0,10.56,elements,12.16,Agita ABELE,LAT,3.00
4,ISU European Figure Skating Championships 2017,LADIES SHORT PROGRAM,b639d77459,Evgenia MEDVEDEVA,RUS,1,78.92,42.00,36.92,0.0,6e36e62b05,CCoSp4,6.0,3.50,elements,4.50,Agita ABELE,LAT,2.00
5,ISU European Figure Skating Championships 2017,LADIES SHORT PROGRAM,b639d77459,Evgenia MEDVEDEVA,RUS,1,78.92,42.00,36.92,0.0,8f7180d180,2A,5.0,3.63,elements,4.63,Agita ABELE,LAT,2.00
6,ISU European Figure Skating Championships 2017,LADIES SHORT PROGRAM,b639d77459,Evgenia MEDVEDEVA,RUS,1,78.92,42.00,36.92,0.0,9158f70d60,Composition,NaN,NaN,components,9.21,Agita ABELE,LAT,9.25
7,ISU European Figure Skating Championships 2017,LADIES SHORT PROGRAM,b639d77459,Evgenia MEDVEDEVA,RUS,1,78.92,42.00,36.92,0.0,9d6f320ec5,Interpretation of the Music,NaN,NaN,components,9.36,Agita ABELE,LAT,9.50
8,ISU European Figure Skating Championships 2017,LADIES SHORT PROGRAM,b639d77459,Evgenia MEDVEDEVA,RUS,1,78.92,42.00,36.92,0.0,a85b51a8e7,Performance,NaN,NaN,components,9.36,Agita ABELE,LAT,9.50
9,ISU European Figure Skating Championships 2017,LADIES SHORT PROGRAM,b639d77459,Evgenia MEDVEDEVA,RUS,1,78.92,42.00,36.92,0.0,c07f5eb3da,FCSp4,1.0,3.20,elements,4.20,Agita ABELE,LAT,2.00


In [33]:
# rinomino le colonne per coerenza con la presentazione
df.rename(columns={'competition':'event'}, inplace=True)

In [35]:
df.to_csv('final.csv',index=False)